# CIC-IDS-2017 - Pré-Processamento 2

## Importando Bibliotecas

In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

## Carregando os Dados

In [17]:
cicids =  pd.read_csv('../datasets-sem-tratamento/cic_ids_2017.csv')
train = pd.read_csv('../datasets-sem-tratamento/cic_ids_2017_train.csv')
test =  pd.read_csv('../datasets-sem-tratamento/cic_ids_2017_test.csv')

### Definir se vai ser usado o conjunto completo ou a divisão já feita entre treino e teste

In [18]:
datasets = [cicids]
#datasets = [train, test]

In [19]:
# Concatenando-os em apenas um dataframe
df = pd.concat(datasets, ignore_index=True)
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830738,53,32215,4,2,112,152,28,28,28.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830739,53,324,2,2,84,362,42,42,42.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830740,58030,82,2,1,31,6,31,0,15.5,21.92031,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830741,53,1048635,6,2,192,256,32,32,32.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


## Pré-Processamento

### Padronização dos nomes dos atributos

In [20]:
df.columns = (
df.columns.str.strip()
            .str.lower()
            .str.replace(' ', '_', regex=False)
            .str.replace('\(', '', regex=True)
            .str.replace('\)', '', regex=True)
)

### Tratamento de valores nulos e infinitos

In [21]:
# Substituindo valores nulos por 0
df['flow_bytes/s'] = df['flow_bytes/s'].replace([np.nan,",,"],0)
df['flow_packets/s'] = df['flow_packets/s'].replace([np.nan,",,"],0)

# Substituindo valores infinitos por -1
df['flow_bytes/s'] = df['flow_bytes/s'].replace([np.inf,",,"],-1)
df['flow_packets/s'] = df['flow_packets/s'].replace([np.inf,",,"],-1)

df.loc[df['flow_bytes/s'] == "Infinity", 'flow_bytes/s'] = -1
df.loc[df['flow_packets/s'] == "Infinity", 'flow_packets/s'] = -1

df['flow_bytes/s'] = df['flow_bytes/s'].apply(pd.to_numeric) 
df['flow_packets/s'] = df['flow_packets/s'].apply(pd.to_numeric)

### Gerando um arquivo para o arquivo pré-processado

In [22]:
df.to_csv("../datasets-tratados/cic_ids_2017_pp2_processed.csv", index=False)

### Normalização

In [23]:
# Separando atributos e rótulos
X = df.drop('label', axis=1)
y = df['label']

# Inicializando o MinMaxScaler
scaler = MinMaxScaler()

# Ajustando o scaler aos dados e transformando os dados
dados_normalizados = scaler.fit_transform(X)

# Transformando em DataFrame
df_normalizado = pd.DataFrame(dados_normalizados, columns=X.columns)

df_normalizado['label'] = df['label']

c:\Users\rafac\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\rafac\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\rafac\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\rafac\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning:

### Gerando um arquivo para o dataset normalizado

In [24]:
df_normalizado.to_csv("../datasets-tratados/cic_ids_2017_pp2_normalized.csv", index=False)

### Removendo duplicatas

In [25]:
print(f"{df_normalizado.duplicated().sum()} duplicatas no dataframe")

308385 duplicatas no dataframe


In [26]:
df_normalizado.drop_duplicates(inplace=True)
df_normalizado

,destination_port,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,...,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,label
0,0.837186,1.333333e-07,0.000005,0.000000,9.302326e-07,0.000000e+00,0.000242,0.002581,0.001010,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,0.840070,1.016667e-06,0.000000,0.000003,4.651163e-07,9.153974e-09,0.000242,0.002581,0.001010,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,0.840085,5.416666e-07,0.000000,0.000003,4.651163e-07,9.153974e-09,0.000242,0.002581,0.001010,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,0.705516,3.916666e-07,0.000000,0.000003,4.651163e-07,9.153974e-09,0.000242,0.002581,0.001010,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,0.837156,1.333333e-07,0.000005,0.000000,9.302326e-07,0.000000e+00,0.000242,0.002581,0.001010,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830738,0.000809,2.685666e-04,0.000014,0.000007,8.682171e-06,2.319007e-07,0.001128,0.012043,0.004713,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2830739,0.000809,2.808333e-06,0.000005,0.000007,6.511628e-06,5.522898e-07,0.001692,0.018065,0.007070,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2830740,0.885481,7.916666e-07,0.000005,0.000003,2.403101e-06,9.153974e-09,0.001249,0.000000,0.002609,0.003076,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2830741,0.000809,8.738733e-03,0.000023,0.000007,1.488372e-05,3.905696e-07,0.001289,0.013763,0.005386,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


## Verificando o dataset tratado

In [27]:
df_normalizado

,destination_port,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,...,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,label
0,0.837186,1.333333e-07,0.000005,0.000000,9.302326e-07,0.000000e+00,0.000242,0.002581,0.001010,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,0.840070,1.016667e-06,0.000000,0.000003,4.651163e-07,9.153974e-09,0.000242,0.002581,0.001010,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,0.840085,5.416666e-07,0.000000,0.000003,4.651163e-07,9.153974e-09,0.000242,0.002581,0.001010,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,0.705516,3.916666e-07,0.000000,0.000003,4.651163e-07,9.153974e-09,0.000242,0.002581,0.001010,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,0.837156,1.333333e-07,0.000005,0.000000,9.302326e-07,0.000000e+00,0.000242,0.002581,0.001010,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830738,0.000809,2.685666e-04,0.000014,0.000007,8.682171e-06,2.319007e-07,0.001128,0.012043,0.004713,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2830739,0.000809,2.808333e-06,0.000005,0.000007,6.511628e-06,5.522898e-07,0.001692,0.018065,0.007070,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2830740,0.885481,7.916666e-07,0.000005,0.000003,2.403101e-06,9.153974e-09,0.001249,0.000000,0.002609,0.003076,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2830741,0.000809,8.738733e-03,0.000023,0.000007,1.488372e-05,3.905696e-07,0.001289,0.013763,0.005386,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [28]:
# Valores por classe
df_normalizado['label'].value_counts()

label
BENIGN                        2096480
DoS Hulk                       172849
DDoS                           128016
PortScan                        90819
DoS GoldenEye                   10286
FTP-Patator                      5933
DoS slowloris                    5385
DoS Slowhttptest                 5228
SSH-Patator                      3219
Bot                              1953
Web Attack � Brute Force         1470
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64

## Treino e Teste

In [29]:
if len(datasets) == 1:
    train, test = train_test_split(df_normalizado, test_size=0.3, random_state=42)
elif len(datasets) == 2:    
    train = df_normalizado.head(round(0.7 * len(df_normalizado)))
    test = df_normalizado.tail(round(0.3 * len(df_normalizado)))
else:
    print('Erro')

## Criando os arquivos .csv para os datasets

In [30]:
if len(datasets) == 1:
    train.to_csv("../datasets-tratados/cic_ids_2017_pp2_train.csv", index=False)
    test.to_csv("../datasets-tratados/cic_ids_2017_pp2_test.csv", index=False)
elif len(datasets) == 2:    
    train.to_csv("../datasets-tratados/cic_ids_2017_pp2_train.csv", index=False)
    test.to_csv("../datasets-tratados/cic_ids_2017_pp2_test.csv", index=False)
else:
    print('Erro')